**Chapter 6 - Other Flowables**

ReportLab has several other Flowables you can use besides the
Paragraph and Table that we covered in the last couple of
chapters. In this chapter we will look at the following Flowables (important Flowables in bold italics; many of these are minor and/or not frequently used):
- Preformatted
- XPreformatted
- ***Image***
- ***Spacer***
- ***PageBreak***
- CondPageBreak (so rarely used that it is not covered in this chapter)
- KeepTogether
- TableOfContents
- SimpleIndex
- ListFlowable

**Preformatted**

ReportLab has a couple of Flowables that are somewhat related called
`Preformatted` and `XPreformatted`. The `Preformatted` Flowable is
described as being kind of like the HTML `<PRE>` tag. According to
ReportLab’s docstring:
>It attempts to display text exactly as you typed it in a fixed width “typewriter” font. By default the line breaks are exactly where you
put them, and it will not be wrapped. You can optionally define a
maximum line length and the code will be wrapped; and extra characters
to be inserted at the beginning of each wrapped line (e.g. ‘> ‘).

The class’s instantiation looks like this:
```python
Preformatted(text, style, bulletText=None,
dedent=0, maxLineLength=None, splitChars=None,
newLineChars=None)
```
See book page 164 for more details - Michael Driscoll notes that he pretty much never uses these Flowables, opting for `Paragraph` instead.

**The Image Flowable**

We have actually used the `Image` Flowable in previous chapters, but
here we will go over it again in a bit more detail. Here is what it
takes to create an `Image` in ReportLab:
```python
Image(filename, width=None, height=None)
```
As you can see, the Image class takes 3 arguments: The filename, the
width and the height of the image. The `filename` argument is required
and can be a file path, a file-like object or an instance of
`reportlab.graphics.shapes.Drawing`. Be default, ReportLab only
supports the *jpeg* format. However if you have the `Pillow` (or PIL)
package installed, then most other image types are also supported. The
`width` and `height` parameters specify the dimensions of the image
in points. If you do not specify one of the parameters, then ReportLab
will just assume that the other dimension of the image is in points too
and use it as is. What this means is that if you only specify one of the
dimensions, the aspect ratio of the image will not be maintained and the
image will be stretched.

In [1]:
# scaled_image.py

from reportlab.lib import utils
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Image, SimpleDocTemplate

img_filepath = r"C:\Users\lucy\OneDrive - Aldatu Biosciences\Desktop\PANDAA qPCR Results\vhf\aldatulogo_icon.gif"

def scaled_image(desired_width):
    doc = SimpleDocTemplate("image_with_scaling.pdf", pagesize=letter)
    story = []

    img = utils.ImageReader(img_filepath) #ImageReader uses Pillow to get information about image, so that we can grab the image size
    img_width, img_height = img.getSize()
    aspect = img_height / float(img_width) #calculate aspect ratio based on obtained height and width information
    img = Image(img_filepath,
                width=desired_width,
                height=(desired_width * aspect)) #scale height based on aspect ratio
    img.hAlign = 'CENTER'
    story.append(img)
    doc.build(story)

if __name__ == '__main__':
    scaled_image(50)

**The Spacer Flowable**

The Spacer Flowable has been mentioned in some of our previous chapters
as well. Here is a reminder of how you can create one:
```python
Spacer(width, height)
```
Note that the `width` parameter is ignored currently and has been for
many years. The only use of the Spacer is for adding vertical space that
is useful for positioning other Flowables on the page.

In [2]:
# spacer_demo.py

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

def no_spacers():

    for p in range(3):
        text = "<para align=center>Hello, I'm a Paragraph</para>"
        para = Paragraph(text, style=styles["Normal"])
        flowables.append(para)
    
def use_spacers():
    
    for p in range(3):
        text = "<para align=center>Hello, I'm a Paragraph</para>"
        para = Paragraph(text, style=styles["Normal"])
        flowables.append(para)
        spacer = Spacer(width=0, height=50)
        flowables.append(spacer)

if __name__ == '__main__':
    doc = SimpleDocTemplate("spacers_examples.pdf",
                            pagesize=letter
                           )
    styles = getSampleStyleSheet()
    flowables = []
    no_spacers()
    use_spacers()
    doc.build(flowables)

**PageBreak**

We mentioned the `PageBreak` Flowable back in chapter 3 and as its
name implies, it will add a page break to your document by consuming all
the vertical space left on the page. This will cause any other Flowables
that are added after the PageBreak to appear on the following page. Note
that this doesn’t apply if you try to add a PageBreak to a Frame. In
that case, ReportLab would detect that as a Frame break and the Flowable
would continue in the next Frame, which could be on the same page. The
`BaseDocTemplate` will detect PageBreaks though, so when using them in
a template, they should work as expected.

In [ ]:
# page_break.py

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, PageBreak
from reportlab.lib.styles import getSampleStyleSheet


def page_break():

    doc = SimpleDocTemplate("page_break.pdf")
    styles = getSampleStyleSheet()
    flowables = []

    text = "Hello, I'm a Paragraph"
    para = Paragraph(text, style=styles["Normal"])
    flowables.append(para)

    pagebreak = PageBreak()
    flowables.append(pagebreak)

    text = "Hello, I'm a Paragraph on page 2"
    para = Paragraph(text, style=styles["Normal"])
    flowables.append(para)
    
    doc.build(flowables)


if __name__ == '__main__':

    page_break()

**KeepTogether**

The `KeepTogether` Flowable is a class that takes a list of other
Flowables and will attempt to keep the entire list in the same
Frame.
```python
KeepTogether(flowables)
```
If the list of Flowables exceeds the height of the Frame in which they
are contained, the a frame break will occur.

**How to Add a Table of Contents**

ReportLab has a neat Flowable called `TableOfContents` that will allow
you to add a Table of Contents to your PDF. To create a
`TableOfContents` instance, all you need to do is the following:
```python
from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus.tableofcontents import TableOfContents
toc = TableOfContents()
```
The TableOfContents Flowable has built-in styles, but from looking
at the documentation, you will usually want to override those and
replace them with your own. You can do that like this:
```python
from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus.tableofcontents import TableOfContents
toc = TableOfContents()

toc.levelStyles = [
    ParagraphStyle(name = 'Heading1',
                   fontSize = 16,
                   leading = 16),
    ParagraphStyle(name = 'Heading2',
                   fontSize = 12,
                   leading = 14),
                  ]
```
As you can see, all you need to do is pass in some `ParagraphStyles` to add your
styles. According to the documentation, you can use the
`addEntry` or `AddEntries` methods to add Table of Contents entries
manually. However, after speaking with the ReportLab developers, these
commands need to be done during the PDF rendering process, so using them
directly is nigh impossible. Instead, the preferred method is to create
a custom document template and override its `afterFlowable` method.
Let’s take a look at an example that is based on one from ReportLab’s
user guide (below).
<br></br>
The first thing we do is sub-class `BaseDocTemplate` and set up a few
things in the `__init__`. Then we override the `afterFlowable` method. The key
point here is that we call `self.notify` with a
`‘TOCEntry’` notification string. This will tell our TableOfContents
object that the template has detected an entry that should be added to
the table of contents. In the notification, you will need to pass the
entry text, page number and an optional destination key.
The destination key is a bookmark that should make those items that have
it into clickable links. The rest of the code in this example lies in
the `main` function where we create an instance of our custom
template, add our TableOfContents and various Paragraphs and PageBreaks.
You will note that we need to call the `multiBuild` method on our
document template instance to actually create our TableOfContents. This
is because ReportLab will need to make several passes over the document
to generate your table of contents.

In [11]:
# toc_creator.py

from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import PageBreak
from reportlab.platypus.paragraph import Paragraph
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.platypus.tableofcontents import TableOfContents
from reportlab.platypus.frames import Frame
from reportlab.lib.units import cm

class MyDocTemplate(BaseDocTemplate):
    def __init__(self, filename, **kw):
        self.allowSplitting = 0
        BaseDocTemplate.__init__(self, filename, **kw)
        template = PageTemplate('normal', [Frame(2.5*cm, 2.5*cm, 15*cm, 25*cm, id='F1')])
        self.addPageTemplates(template)

    def afterFlowable(self, flowable):
    # Registers the Table Of Contents entries.
    # Ensures that our doc template monitors added flowables;
    # if the added flowable is a Paragraph, our doc template will read its text and get its style;
    # if the style is Heading1 or Heading2, the text of the paragraph flowable will be registered to the table of contents via TOCEntry
        if flowable.__class__.__name__ == 'Paragraph':
            text = flowable.getPlainText()
            style = flowable.style.name
            if style == 'Heading1':
                self.notify('TOCEntry', (0, text, self.page))           #tuple: (level, text, page) --> TOC level, text to display, page number where Paragraph flowable exists
            if style == 'Heading2':
                self.notify('TOCEntry', (1, text, self.page))
    #the "notify" method here is super useful - can look for paragraphs, but can also look for images/figures, etc.

def main():
    heading1_style = ParagraphStyle(name = 'Heading1',
                                    fontSize = 16,
                                    spaceBefore = 32,
                                    spaceAfter = 24)                      
    heading2_style = ParagraphStyle(name = 'Heading2',
                                    fontSize = 12,
                                    spaceBefore = 16,
                                    spaceAfter = 16)
    # create story and table of contents object
    story = []
    toc = TableOfContents()

    # Set the Paragraph styles in the Table of Contents
    toc.levelStyles = [heading1_style, heading2_style]                  #map styles we made to Heading1, Heading2 in the doc template
    toc.dot = '\u2022'
    story.append(toc)                                                   #table of contents Flowable is added to story as first Flowable
    story.append(PageBreak())

    ipsum = '''Lorem ipsum dolor sit amet, consectetur adipiscing elit,
    sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.
    Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris
    nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in
    reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla
    pariatur. Excepteur sint occaecat cupidatat non proident, sunt in
    culpa qui officia deserunt mollit anim id est laborum.'''

    story.append(Paragraph('Heading #1', heading1_style))
    story.append(Paragraph(ipsum, ParagraphStyle('body')))
    story.append(Paragraph('Sub-heading #1', heading2_style))
    story.append(Paragraph(ipsum, ParagraphStyle('body')))
    story.append(PageBreak())
    story.append(Paragraph('Sub-heading #2', heading2_style))
    story.append(Paragraph(ipsum, ParagraphStyle('body')))
    story.append(Paragraph('Heading #2', heading1_style))

    story.insert(0,                                                     #using list logic, insert a title for TOC as first item in story
                 Paragraph('Table of Contents',
                           ParagraphStyle('TOC_Title',                  #this style is built-in, apparently
                                          fontSize=20,
                                          leading=24
                                         )
                          )
                )

    
    doc = MyDocTemplate('toc.pdf')
    doc.multiBuild(story)                                               #multiBuild is necessary here because of our use of table of contents - doc will need to be "read" multiple times to generate TOC

if __name__ == '__main__':
    main()

**SimpleIndex**

ReportLab has basic support for adding an index to your PDFs as well.
You can accomplish this via the `SimpleIndex` Flowable. To index a
word in ReportLab, you will need to use the `<index>` tag along
with the item attribute, like this:
```python
ptext = """I'm a custom <index item="bulletted"/>bulletted paragraph"""
```
This will tell ReportLab that you would like to add an index for the
word “bulletted”. The other piece that is needed is to create a
`SimpleIndex` instance and add it to the end of your flowables list.

In [14]:
# simple_index.py

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, PageBreak
from reportlab.platypus.tableofcontents import SimpleIndex
from reportlab.lib.styles import getSampleStyleSheet

def simple_index():
    doc = SimpleDocTemplate("simple_index.pdf",
                            pagesize=letter
                           )
    styles = getSampleStyleSheet()
    flowables = []

    ptext = """I'm a custom <index item="bulletted"/>bulletted paragraph"""     #index 'bulletted'
    para = Paragraph(ptext, style=styles["Normal"], bulletText='-')
    flowables.append(para)
    flowables.append(PageBreak())

    ptext = """<index item="Python,word"/>Python is an indexed word"""          #index 'word' under heading 'Python'
    para = Paragraph(ptext, style=styles["Normal"])
    flowables.append(para)

    index = SimpleIndex(dot='.')                                                #optional 'dot' kw - can specify dotted line to connect word to page number (TOC-style) instead of "entry, page" format
    flowables.append(PageBreak())
    flowables.append(index)
    
    doc.build(flowables, canvasmaker=index.getCanvasMaker())                    #makes doc builder use index's canvas maker object

if __name__ == '__main__':
    simple_index()

**ListFlowable / ListItem**

The last Flowables we will be covering in this chapter is the
`ListFlowable` and its companion, the `ListItem`. These Flowables
can be used to make ordered and unordered lists. You can also nest the
lists. The cool thing about the `ListFlowable` is that it can contain
any other Flowable and will use them to create the ordered list. You can
also change the font, color, size, style and position of the list number
or the bullets in unordered lists. You may also change the type of
numbering applied to use lower or upper case letters or upper or lower
Roman numerals via the `bulletType` property.
If you want the list to be unordered, then set `bulletType='bullet'`.
You can also change the default appearance of a ListFlowable by wrapping
them in a ListItem and setting its properties.

In [17]:
# list_flowable_squares.py

from reportlab.lib.pagesizes import letter
from reportlab.platypus import ListFlowable, ListItem
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

def list_flowable_squares():
    doc = SimpleDocTemplate("list_flowable_squares.pdf",
                            pagesize=letter
                           )
    styles = getSampleStyleSheet()
    normal = styles['Normal']
    story = []

    flowables = [
        Paragraph('Paragraph numero uno', normal),
        ListItem(Paragraph('Paragraph #2', normal),
                 bulletColor="blue"), #make bullet/number color for this one blue (does not affect others)
        Paragraph('Paragraph #3', normal),
    ]

    flowables.append(
        ListFlowable( #fourth item in list is a ListFlowable; there will not be text for this list item!
            [Paragraph("I'm a sublist item", normal),
             ListItem(Paragraph("I'm another sublist item", normal),
                      bulletColor='blue'),
             ListItem(Paragraph("I'm the last sublist item", normal),
                      bulletColor='red')
            ],
        bulletType='bullet', #use a shape instead of numbers
        start='square' #change bullet shape; ReportLab also supports small/large circles, discs, diamonds, arrowheads, sparkles, stars, checkboxes
    ))

    lflow = ListFlowable(flowables, bulletType='I') #use capitalized Roman numerals for numbering; note that this is only for high-level bulleting, not the ListFlowable sublist
    story.append(lflow)

    doc.build(story)

if __name__ == '__main__':
    list_flowable_squares()